# Signal Efficiency in the Region of Interest for nEXO
This Juptyer notebook calculates the signal efficiency, i.e. the fraction of $ 0\nu\beta\beta$ events that fall within the region of interest (ROI) as defined below. This number will be used in the counting experiment calculations in this [Jupyter Notebook](https://github.com/nEXO-collaboration/sensitivity/blob/sensitivity2020/work/SensitivityPaper2020_scripts/DiscoveryPotential/Counting_Experiment.ipynb)

### Adjusting cell width

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

### Importing packages

In [2]:
import numpy as np
import uproot # uproot examples: https://indico.cern.ch/event/686641/contributions/2894906/attachments/1606247/2548596/pivarski-uproot.pdf

### Defining cuts
Define cut that selects events that are within the ROI, i.e. passing all standoff quality cuts, the charge-over-light cut, be within the FWHM region around the Q-value as defined in this [Jupyter Notebook](https://github.com/nEXO-collaboration/sensitivity/blob/sensitivity2020/work/SensitivityPaper2020_scripts/DiscoveryPotential/FindEnergyROI.ipynb) and have a DNN value of larger than 0.85 (qualifying as Single-Site).

In [3]:
def GetROICut(tree):
    cut = np.where((tree['passed_xy_thresh'].array() == True) \
               & (tree['passed_z_thresh'].array() == True) \
               & (tree['n_x_ch_abovenoise'].array() > 0) \
               & (tree['n_y_ch_abovenoise'].array() > 0) \
               & (tree['m_nOPCal'].array() < (1.077*tree['m_nQ'].array()+313)) \
               & (tree['m_nOPCal'].array() > (0.597*tree['m_nQ'].array()-216)) \
               & (tree['standoff'].array() > 0.0) \
               & (tree['m_DNNvalue'].array() > 0.85) \
               & (tree['energy'].array() > 2435) \
               & (tree['energy'].array() < 2481) \
               & (tree['NESTBugFound'].array() == False) \
               & (tree['NearAnodeBugFound'].array() == False) \
              )
    return cut        

Define cut that removes events in the denominator that we know are bad due to a bug in NEST

In [4]:
def GetBugCut(tree):
    cut = np.where((tree['NESTBugFound'].array() == False) \
               & (tree['NearAnodeBugFound'].array() == False) \
               & (tree['standoff'].array() > 0.0) \
               )
    return cut         

### Reading in data from ROOT files

In [5]:
bb0n = uproot.open("/p/vast1/nexo/data/merged-v10b/Baseline2019_bb0n_FullLXe.root")['tree']

### Applying cuts

In [6]:
AllEvents = bb0n['energy'].array()[GetBugCut(bb0n)]

In [7]:
ROIEvents = bb0n['energy'].array()[GetROICut(bb0n)]

### Calculating the signal efficiency 

In [8]:
print('Number of total events: %d' % len(bb0n['energy'].array()))
print('Number of total events (after NestBug): %d' % len(AllEvents))
print('Number of events passing all cuts and are within ROI: %d' % len(ROIEvents))
print('ROI signal efficiency: %.2f' % (len(ROIEvents)/len(AllEvents)*100))

Number of total events: 75098801
Number of total events (after NestBug): 74007964
Number of events passing all cuts and are within ROI: 42598484
ROI signal efficiency: 57.56
